# Plots
> Plotting utilities

In [ ]:
#| default_exp plots

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, numpy as np, pandas as pd, seaborn as sns
import matplotlib, matplotlib.pyplot as plt, matplotlib.animation as animation
import networkx as nx
from gode.graph import get_attn_edges_at_t, get_attn_mat_at_t

def facet_plot_features_over_time(
    df:pd.DataFrame,
    col:str='tf',
    row:str='cell_type',
    hue:str='type',
    x:str='time',
    y:str='expression'
):
    g = sns.FacetGrid(
        df, 
        col=col, row=row, hue=hue, 
        sharex=True,
        sharey=True,
    )
    g.map(sns.lineplot, x, y)
    g.add_legend()
    return g

In [ ]:
#| export

def animate_nx_example(
    graph, pos=None, filename=None,
    frames:int=10, interval:int=500, repeat:bool=False
):

    if pos is None:
        pos = nx.spring_layout(graph)

    node_colors = [i for i, node in enumerate(graph.nodes())]
    _min = np.min(node_colors)
    _max = np.max(node_colors)

    cmap = plt.get_cmap('viridis', int(_max - _min) * 100)
    node_colors = list(map(lambda n: cmap(n), node_colors)) 

    # NOTE: using inner function to have access to the graph, pos, and node_colors
    # without having to redefine them
    def animation_function_example(i):
        # NOTE: move node_colors here if you want them to update per frame i
        nx.draw(
            graph, pos=pos, node_colors=node_colors,
            arrows=True, with_labels=True, node_size=500, font_size=8
        )

    fig = plt.gcf()
    fig_animated = animation.FuncAnimation(fig, animation_function_example, frames, interval, repeat)
    if filename is not None:
        fig_animated.save(os.path.expanduser(os.path.abspath(filename)))

    return fig

In [ ]:
#| export

def custom_features_over_time(
    df:pd.DataFrame,
    df_corr:pd.DataFrame,
    col:str='tf',
    row:str='cell_type',
    hue:str='type',
    x:str='time',
    y:str='expression'
):
    grid_figsize = [4, 3]
    
    
    cols = np.sort(df[col].unique())
    rows = np.sort(df[row].unique())
    hues = np.sort(df[hue].unique())
    n_cols = cols.size
    n_rows = rows.size

    dpi = 300
    grid_figsize = (grid_figsize[0] * n_cols, grid_figsize[1] * n_rows)
    fig = plt.figure(None, grid_figsize, dpi=dpi)

    hspace = 0.3
    wspace = None
    gspec = plt.GridSpec(n_rows, n_cols, fig, hspace=hspace, wspace=wspace)


    cmap = plt.get_cmap('tab10')

    axs = []
    for g, gs in enumerate(gspec):
        for i, r in enumerate(rows):
            for j, c in enumerate(cols):
                ax = plt.subplot(gspec[i, j])

                scors = []
                for k, h in enumerate(hues):                
                    df_cur = df[(df[col] == c) & (df[row] == r) & (df[hue] == h)]
                    ax.plot(df_cur[x], df_cur[y], color=cmap(k), label=h)
                    xmin, xmax = ax.get_xaxis().get_data_interval()
                    ymin, ymax = ax.get_yaxis().get_data_interval()

                    if k == len(hues) - 1:
                        scor = df_corr.loc[c, r].round(3)
                        s = f'corr: {scor}'
                        ax.text(
                            xmin+(xmax-xmin)*0.05, ymin+(ymax-ymin)*0.1, s, fontsize=10,
                            bbox={'facecolor': 'lightblue', 'alpha': 1, 'pad': 10,}
                        )
                        ax.set_title(f'{r} | {c}')
                        if i == len(rows) - 1:
                            ax.set_xlabel(x.capitalize())
                        else:
                            ax.xaxis.set_ticks([])

                        if j == 0:
                            ax.set_ylabel(y.capitalize())

    handles, labels = ax.get_legend_handles_labels()
    gspec.figure.legend(handles[:2], labels[:2], loc='center right')
    suptitle = fig.suptitle(
        f'{y.capitalize()} over {x} for various {col} across {row}', 
        fontsize=20, 
        bbox={'pad':20,  'facecolor': None, 'alpha':0}
    )
    return fig

In [ ]:
#| export
from typing import Union
# NOTE: this plots the full graph over t where edges are colored based off of attention
def animate_attn_example(
    graph, pos=None, attns=None, filename=None,
    frames:int=10, interval:int=500, repeat:bool=False, 
    seed:int=3, node_colors:Union[list,str]='gray', fps=15,
    node_map=None, edge_map=None
):

    # Generate single position to keep graph consistent over animation
    if pos is None:
        pos = nx.spring_layout(graph, seed=seed)

    # No node colors provided, try to get them from node data
    if node_colors is None:
        node_colors = nx.get_node_attributes(graph, 'color').values()
    
    # Graph doesn't have explicit colors for nodes
    if len(node_colors) == 0:
        node_colors = [i for i, node in enumerate(graph.nodes())]
        _min = np.min(node_colors)
        _max = np.max(node_colors)

        cmap = plt.get_cmap('viridis', int(_max - _min) * 100)
        node_colors = list(map(lambda n: cmap(n), node_colors)) 

    inode_map = None
    if node_map is not None:
        inode_map = {v:k for k,v in node_map.items()}
        
    
    frames = attns.shape[0]
    attn_cmap = plt.get_cmap('magma')
    
    fig = plt.gcf()
            
    # NOTE: using inner function to have access to the graph, pos, and node_colors
    # without having to redefine them
    def animation_function_example(i):
        # NOTE: move node_colors here if you want them to update per frame i
        ax = fig.gca()        
        attn = attns[i, :]
        edge_colors = list(map(attn_cmap, attn))
        nx.draw(
            graph, pos=pos, 
            node_color=node_colors,
            edge_color=edge_colors,
            arrows=True, with_labels=False, node_size=100, font_size=8,
            ax=ax
        )
        if inode_map is not None:
            nx.draw_networkx_labels(graph, pos, inode_map, font_size=8, ax=ax)
        
        return ax

    fig_animated = animation.FuncAnimation(
        fig, animation_function_example, 
        frames=frames, interval=interval, repeat=repeat, # blit=False
    )
    if filename is not None:
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(
            fps=fps, metadata=dict(artist='gode'), bitrate=1800
        )
        fig_animated.save(os.path.expanduser(os.path.abspath(filename)), writer=writer)

    return fig_animated

In [ ]:
#| export

# NOTE: this plots the adjacency matrix of the given graph
# where edges drawn are based off the attention over t
def animate_attn_adj_example(
    graph, attns=None, filename=None,
    frames:int=10, interval:int=500, repeat:bool=False, 
    seed:int=3, fps=15, node_map=None, edge_map=None
):
   
    frames = attns.shape[0]
    
    nodes = list(graph.nodes())
    fig = plt.gcf()
#     fig.set_size_inches(12, 12)
    # NOTE: using inner function to have access to the graph, pos, and node_colors
    # without having to redefine them
    def animate_adj(i):
        ax = plt.gca()
        ax.clear()     
        
        
        # NOTE: if use average attention
        # attn = attns.mean(axis=0).reshape(1, -1)
        # edge_list = get_attn_edges_at_t(graph, attn, 0, 0)    
        # adj_mat = np.round(get_attn_mat_at_t(graph, attn, 0), 6)
        # attn = attns[i, :][attns[i, :] >= 0]
        
        # NOTE: if not using average attention
        adj_mat = np.round(get_attn_mat_at_t(graph, attns, i, edge_map), 6)
        
        im = ax.imshow(adj_mat)
        ax.tick_params(direction='out', axis='both', which='major', pad=5)
        # check if colorbar already exists
        if any(list(map(lambda e : e.get_label() == '<colorbar>', ax.figure.axes))):
            pass
        else:
            cbar = fig.colorbar(im)

        if node_map is not None:
            vals = list(node_map.values())[0:-1:2]
            keys = list(node_map.keys())[0:-1:2]
            ax.set_xticks(vals)
            ax.set_yticks(vals)
            ax.set_xticklabels(keys, rotation=45, fontsize='xx-small', va='top', ha='right')
            ax.set_yticklabels(keys, rotation='horizontal', fontsize='xx-small')
        return ax,

    fig_animated = animation.FuncAnimation(
        fig, animate_adj, 
        frames=frames, interval=interval, repeat=repeat, # blit=False
    )
    
    if filename is not None:
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(
            fps=fps, metadata=dict(artist='gode'), bitrate=1800
        )
        fig_animated.save(os.path.expanduser(os.path.abspath(filename)), writer=writer)

    return fig_animated

In [ ]:
#| export

# NOTE: this plots the edges from the given graph of the attention at time T
# values surpase a given and fixed threshold, where these are iterated over t
def animate_attn_edge_example(
    graph, pos=None, attns=None, threshold=0.15, filename=None,
    frames:int=10, interval:int=500, repeat:bool=False, 
    seed:int=3, node_colors:Union[list,str]='gray', fps=15, node_map=None
):

    # Generate single position to keep graph consistent over animation
    if pos is None:
        pos = nx.spring_layout(graph, seed=seed)

    # No node colors provided, try to get them from node data
    if node_colors is None:
        node_colors = nx.get_node_attributes(graph, 'color').values()
    
    # Graph doesn't have explicit colors for nodes
    if len(node_colors) == 0:
        node_colors = [i for i, node in enumerate(graph.nodes())]
        _min = np.min(node_colors)
        _max = np.max(node_colors)

        cmap = plt.get_cmap('viridis', int(_max - _min) * 100)
        node_colors = list(map(lambda n: cmap(n), node_colors)) 

    inode_map = None    
    if node_map is not None:
        inode_map = {v:k for k, v in node_map.items()}
        
    frames = attns.shape[0]
    attn_cmap = plt.get_cmap('magma')
    
    fig = plt.gcf()
            
    # NOTE: using inner function to have access to the graph, pos, and node_colors
    # without having to redefine them
    def animation_function_example(i):
        # NOTE: move node_colors here if you want them to update per frame i
        ax = fig.gca()    
        
        edge_list = get_attn_edges_at_t(graph, attns, i, threshold)        
        attn = attns[i, :][attns[i, :] >= threshold]
        edge_colors = list(map(attn_cmap, attn))
        
        nx.draw_networkx_nodes(
            graph, pos, node_size=100, 
            node_color=node_colors, ax=ax
        )
        if inode_map is not None:
            nx.draw_networkx_labels(graph, pos, inode_map, font_size=8, ax=ax)
        
        nx.draw_networkx_edges(
            graph, pos, edgelist=edge_list, edge_color=edge_colors, ax=ax
        );
        return fig,

    fig_animated = animation.FuncAnimation(
        fig, animation_function_example, 
        frames=frames, interval=interval, repeat=repeat, blit=True
    )
    
    if filename is not None:
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(
            fps=fps, metadata=dict(artist='gode'), bitrate=1800
        )
        fig_animated.save(os.path.expanduser(os.path.abspath(filename)), writer=writer)

    return fig_animated

In [ ]:
#| export

# NOTE: this plots the edges from the given graph of the MEAN attention
# values surpase a given threshold, where the threshold is iterated 
# from the 0 to the max of the mean. 
def animate_mean_attn_edge_example(
    graph, pos=None, attns=None, n_thresholds=10, filename=None,
    frames:int=10, interval:int=500, repeat:bool=False, 
    seed:int=3, node_colors:Union[list,str]='gray', fps=15, node_map=None
):

    # Generate single position to keep graph consistent over animation
    if pos is None:
        pos = nx.spring_layout(graph, seed=seed)

    # No node colors provided, try to get them from node data
    if node_colors is None:
        node_colors = nx.get_node_attributes(graph, 'color').values()
    
    # Graph doesn't have explicit colors for nodes
    if len(node_colors) == 0:
        node_colors = [i for i, node in enumerate(graph.nodes())]
        _min = np.min(node_colors)
        _max = np.max(node_colors)

        cmap = plt.get_cmap('viridis', int(_max - _min) * 100)
        node_colors = list(map(lambda n: cmap(n), node_colors)) 

    inode_map = None    
    if node_map is not None:
        inode_map = {v:k for k, v in node_map.items()}
    
    thresholds = np.round(np.linspace(0, attns.mean(axis=0).max(), n_thresholds), 3) 
    frames = len(thresholds)
    attn_cmap = plt.get_cmap('magma')
    
    fig = plt.gcf()
            
        
    # NOTE: using inner function to have access to the graph, pos, and node_colors
    # without having to redefine them
    def animation_function_example(i):
        # NOTE: move node_colors here if you want them to update per frame i
        ax = fig.gca()    
        ax.clear()
        threshold = thresholds[i]
        attn = attns.mean(axis=0).reshape(1, -1)
        edge_list = get_attn_edges_at_t(graph, attn, 0, threshold)        
        attn = attns[i, :][attns[i, :] >= threshold]
        edge_colors = list(map(attn_cmap, attn))
        ax.set_title(f'Edges where attn >= {threshold}')
        nx.draw_networkx_nodes(
            graph, pos, node_size=100, 
            node_color=node_colors,  ax=ax
        )
        if inode_map is not None:
            nx.draw_networkx_labels(graph, pos, inode_map, font_size=8, ax=ax)
            
        ax = nx.draw_networkx_edges(
            graph, pos, edgelist=edge_list, edge_color=edge_colors, ax=ax
        );
        return fig

    fig_animated = animation.FuncAnimation(
        fig, animation_function_example, 
        frames=frames, interval=interval, repeat=repeat, blit=True
    )
    
    if filename is not None:
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(
            fps=fps, metadata=dict(artist='gode'), bitrate=1800
        )
        fig_animated.save(os.path.expanduser(os.path.abspath(filename)), writer=writer)

    return fig_animated

In [ ]:
#| export


'''
Example usage:

q, base_q = 80, 80
attn_mu = attns.mean(axis=0)
norm_attn = get_norm_attention_at_q(attn_mu, q, base_q)
edge_colors = get_attention_colors_at_q(attn_mu, norm_attn=norm_attn)
edge_list = get_attn_edges_at_t(graph, attn_mu.reshape(1, -1), t=0, threshold=np.percentile(attn_mu, q))   
'''

def get_norm_attention_at_q(
    attn, quartile=90, base_quartile=0,
    cmin=None, cmax=None,
):
    base_threshold  = np.percentile(attn, base_quartile)
    
    if cmin is None:
        cmin = attn[attn >= base_threshold].min()
        
    if cmax is None:
        cmax = attn[attn >= base_threshold].max()
        
    
    norm_fn = matplotlib.colors.Normalize(cmin, cmax)
    
    threshold = np.percentile(attn, quartile)
    norm_attn = norm_fn(attn[attn >= threshold])
    return norm_attn

def get_attention_colors_at_q(
    attn, quartile=90, base_quartile=0,
    cmap=None, cmin=None, cmax=None, norm_attn=None,
    normalize=True
):
    
    if cmap is None:
        cmap = plt.get_cmap('viridis_r')
    
    if norm_attn is None and normalize:
        norm_attn = get_norm_attention_at_q(attn, quartile, base_quartile, cmin, cmax)
        
    if not normalize:
        norm_attn = attn
    
    edge_colors = list(map(cmap, norm_attn))
    return edge_colors

def make_edge_colorbar(attn, base_quartile, cmap=None, cmin=None, cmax=None, round=3):
    base_threshold  = np.percentile(attn, base_quartile)
    
    if cmap is None:
        cmap = plt.get_cmap('viridis_r')

    if cmin is None:
        cmin = attn[attn >= base_threshold].min()
        
    if cmax is None:
        cmax = attn[attn >= base_threshold].max()
        
    
    norm_fn = matplotlib.colors.Normalize(np.round(cmin, round), np.round(cmax, round))
    return plt.colorbar(matplotlib.cm.ScalarMappable(norm_fn, cmap))

In [ ]:
#| export

from gode.utils import make_imap, dict_imap_if_in, all_imappable
def plot_edges_of_mean_attn_at_q(
    ax, ntx_g, ref_g, ref_pos,
    attn, q,         
    node_map, node_map_full,  
    base_q=0,
    inode_map=None, inode_map_full=None,
    cmap=None, colorbar=False, node_size=100,
    font_size=8
):
    if cmap is None:
        cmap = plt.get_cmap('viridis_r')
        
    
    if inode_map is None:
        inode_map = make_imap(node_map)
        
    if inode_map_full is None:
        inode_map_full = make_imap(node_map_full)
        
    norm_attn = get_norm_attention_at_q(attn, q, base_q)
    edge_colors = get_attention_colors_at_q(attn, norm_attn=norm_attn)
    edge_list = get_attn_edges_at_t(ntx_g, attn, 0, np.percentile(attn, q))  
    
    node_colors = nx.get_node_attributes(ref_g, 'color').values()
    
    ref_node_idx_list = dict_imap_if_in(node_map_full, node_map)
    ref_edge_idx_list = {
        (u,v): i 
        for i, (u,v) in enumerate(edge_list) 
        if all_imappable([u, v], inode_map_full, node_map)
    }
    
    nx.draw_networkx_nodes(
        ntx_g, ref_pos, node_size=node_size, 
        node_color=node_colors, 
        nodelist=ref_node_idx_list,
        ax=ax,
    )

    nx.draw_networkx_labels(ntx_g, ref_pos, inode_map, font_size=font_size, ax=ax)
    nxe = nx.draw_networkx_edges(
        ntx_g, ref_pos, node_size=node_size,
        arrows=True, arrowsize=1.1,
        width=np.exp((norm_attn + 0.0) * 1) / 2,        
        edgelist=ref_edge_idx_list,
        edge_color=edge_colors,
        ax=ax
    );
    
    if colorbar:
        cbar = make_edge_colorbar(attn, base_q)
    return ax

In [ ]:
#| export

def make_mean_attn_ref_fig(
    attns,
    ntx_g, ref_g, ref_pos,
    node_map, node_map_full,
    qs:list = [75, 80, 95, 99.5],
    base_q:float=None,
    inode_map=None,
    grid_scale:float=4,
    font_size:int = 12,
    node_size:int = 100,
    epoch:int=9,
):
    
    # Setup
    epoch = 9
    attn_mu = attns.mean(axis=0)
    qs = [75, 80, 95, 99.5]
    
    if base_q is None:
        base_q = qs[0]
        
    if inode_map is None:
        inode_map = make_imap(node_map)
        
    grid_figsize = np.array([4, 3]) / grid_scale

    n_rows = 1
    n_cols = 1 + len(qs)
    s_factor = (n_rows + n_cols)

    dpi = 300
    grid_figsize = (grid_figsize[0] * n_cols * s_factor, grid_figsize[1] * n_rows * s_factor)

    fig = plt.figure(None, grid_figsize, dpi=dpi)

    hspace = 0.3
    wspace = None

    gspec = plt.GridSpec(
        n_rows, n_cols, fig, hspace=None, wspace=wspace,
        width_ratios=[grid_figsize[0] for col in range(n_cols)],
        height_ratios=[grid_figsize[1] for row in range(n_rows)],
    )


    # (1) Plot Ground Truth Transcription Factor Network
    ax = plt.subplot(gspec[0, 0])

    node_colors = nx.get_node_attributes(ref_g, 'color').values()
    nx.draw_networkx_nodes(
        ref_g, ref_pos, node_size=node_size, 
        node_color=node_colors, ax=ax,
    )
    nx.draw_networkx_labels(ref_g, ref_pos, inode_map, font_size=font_size, ax=ax)
    nx.draw_networkx_edges(
        ref_g, ref_pos, node_size=node_size, width=1,
        ax=ax,    
    );
    ax.set_title(f'Ground Truth TF module network')


    # (2) Plot Ground Truth Transcription Factor Network
    ntx_pos = nx.kamada_kawai_layout(ntx_g, scale=1, dim=2)
    node_colors = nx.get_node_attributes(ntx_g, 'color').values()
    for i in range(1, n_cols):
        ax = plt.subplot(gspec[0, i])

        plot_edges_of_mean_attn_at_q(
            ax, ntx_g, ref_g, ref_pos,
            attn_mu, q=qs[i-1], base_q=base_q, node_map=node_map, node_map_full=node_map_full,
            colorbar=(i == n_cols - 1), node_size=node_size, font_size=font_size
        )
        ax.set_title(f'{qs[i-1]}-th percentile of edges')



    fig.suptitle(
        f'Mean attention across time at epoch {epoch}',
        fontsize=20, y=1.05,
        bbox={'pad':20,  'facecolor': None, 'alpha':0}
    );
    return fig

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()